In [ ]:
__version__ = '1.3.1'
__author__ = 'Martino Pulici'

In [ ]:
import pandas as pd
from pgmpy.inference import VariableElimination
from pgmpy.models import BayesianModel
from pgmpy.sampling import BayesianModelSampling

from spotify_audio_features.functions import *

In [ ]:
# Number of graph bins
BINS = 4

In [ ]:
# Data
df = pd.read_csv('georgieva_suta_burton_data.csv', sep=',')
df = df.rename(columns={'Label': 'Chart'})

# Cycle unwanted labels
for label in ['Artist', 'Track', 'Month', 'Key', 'Mode']:
    # Delete data label
    del df[label]
    
# Cycle labels
for label in [
    'Danceability',
    'Valence',
    'Acousticness',
    'Loudness',
    'Instrumentalness',
    'Year',
    'Energy',
    'Tempo',
    'Liveness',
        'Speechiness']:
    # Create bins
    df[label] = pd.cut(df[label], BINS, labels=range(BINS))

In [ ]:
# Create Bayesian model
model = BayesianModel([('Danceability', 'Valence'),
                       ('Acousticness', 'Loudness'),
                       ('Instrumentalness', 'Loudness'),
                       ('Year', 'Loudness'),
                       ('Energy', 'Valence'),
                       ('Tempo', 'Energy'),
                       ('Liveness', 'Energy'),
                       ('Liveness', 'Speechiness'),
                       ('Loudness', 'Energy'),
                       ('Loudness', 'Chart'),
                       ('Speechiness', 'Danceability'),
                       ('Danceability', 'Chart'),
                       ('ArtistScore', 'Chart'),
                       ])

# Fit data to model
model.fit(df)

# Enter if model is incorrect
if not model.check_model():
    print("Incorrect model!")

In [ ]:
# Create CPD
cpd = model.get_cpds('Danceability')
print(cpd)

In [ ]:
# Check active trails
print(model.is_active_trail('Danceability', 'Energy'))
print(model.is_active_trail('Danceability', 'Energy', observed='Speechiness'))

In [ ]:
# Get markov blanket
print(model.get_markov_blanket('Danceability'))

In [ ]:
# Check active trails with markov blanket
print(model.is_active_trail('Danceability', 'Liveness'))
print(model.is_active_trail('Danceability', 'Liveness', observed=model.get_markov_blanket('Danceability')))

In [ ]:
# Create exact inference using variable elimination
exact_inference = VariableElimination(model)
# Create approximate inference using bayesian model
approximate_inference = BayesianModelSampling(model)

In [ ]:
# ArtistScore influence on Chart
exact(exact_inference, 'Chart', {'ArtistScore':0})
exact(exact_inference, 'Chart', {'ArtistScore':1})

In [ ]:
# Loudness influence on Chart
exact(exact_inference, 'Chart', {'Loudness':0})
exact(exact_inference, 'Chart', {'Loudness':1})
exact(exact_inference, 'Chart', {'Loudness':2})
exact(exact_inference, 'Chart', {'Loudness':3})

In [ ]:
# Loudness influence on Energy
exact(exact_inference, 'Energy', {'Loudness':0})
exact(exact_inference, 'Energy', {'Loudness':1})
exact(exact_inference, 'Energy', {'Loudness':2})
exact(exact_inference, 'Energy', {'Loudness':3})

In [ ]:
# Inference example
exact(exact_inference, 'Chart', {'ArtistScore':1,'Danceability':3,'Loudness':3})
exact(exact_inference, 'Chart', {'ArtistScore':0,'Danceability':0,'Loudness':0})

In [ ]:
# Variable and evidence for graphs
VARIABLE = 'Valence'
EVIDENCE_1 = {'Chart': 1}
EVIDENCE_2 = {'Chart': 0}

In [ ]:
# Exact inference
exact(exact_inference, VARIABLE, EVIDENCE_1)

In [ ]:
# Rejection sampling inference
rejection(approximate_inference, VARIABLE, EVIDENCE_1)

In [ ]:
# Likelihood weighted inference
weighted(approximate_inference, VARIABLE, EVIDENCE_1)

In [ ]:
# Exact inference with different evidence
exact(exact_inference, VARIABLE, EVIDENCE_2)

In [ ]:
# Create graph points
sizes, exact_results, rejection_results, weighted_results = graph_points(
    exact_inference, approximate_inference, VARIABLE, EVIDENCE_1)

In [ ]:
# Plot graph
graph(
    sizes,
    exact_results,
    rejection_results,
    weighted_results,
    VARIABLE,
    EVIDENCE_1)

In [ ]:
# Plot differences graph
diff_graph(
    sizes,
    exact_results,
    rejection_results,
    weighted_results,
    VARIABLE,
    EVIDENCE_1)